In [157]:
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(threshold=np.inf)
import dgl
import torch
from torch._C import device
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from dgl.data import MiniGCDataset
from dgl.nn.pytorch import GraphConv
from sklearn.metrics import accuracy_score
import networkx as nx
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [158]:
def MainDate(root, path):
    Original = []
    Target = []
    Label = root[-1]
    Nodes = []
    Edges_Nums = 0
    Edges_Weight = []
    
    # 读取文件并分割每一行，然后转换成浮点数
    adjacency_matrix = np.loadtxt(path, delimiter=' ')

    AM_copy = np.array(adjacency_matrix).copy()
    
    # # 打印读取的邻接矩阵
    # print("AdjacencyMatrix.shape:\n", adjacency_matrix.shape, '\n') 
    # print("AdjacencyMatrix:\n", adjacency_matrix, '\n') 
    
    for i in range(AM_copy.shape[0]):
        if i == 0:
            if  AM_copy[i][i + 1] == 0:
                AM_copy[i][i + 1] = AM_copy[i][i]
                AM_copy[i + 1][i] = AM_copy[i][i]
              
        if i > 0 and i < AM_copy.shape[0] - 1:
            if AM_copy[i + 1][i] == 0 and AM_copy[i][i + 1] == 0:
                AM_copy[i][i + 1] = AM_copy[i][i]
                AM_copy[i + 1][i] = AM_copy[i][i]
        
        if i == AM_copy.shape[0] - 1:
            if  AM_copy[i][i - 1] == 0:
                AM_copy[i][i - 1] = AM_copy[i][i]
                AM_copy[i - 1][i] = AM_copy[i][i]
                
    for i in range(AM_copy.shape[0]):
        Nodes.append(i)            
        for j in range(AM_copy.shape[1]):    
            if AM_copy[i][j] != 0 and i != j:
                Edges_Nums += 1
                Edges_Weight.append(AM_copy[i][j])
                Original.append(i)
                Target.append(j)
    
    return torch.tensor(Original), torch.tensor(Target), torch.tensor(int(Label)), torch.tensor(Nodes), Edges_Nums, torch.tensor(Edges_Weight)

In [159]:
def GraphCreat(path, Original, Target, Label ,Edges_Nums, Edges_Weight):
    # 使用dgl.graph创建图，传入边的列表
    graph = dgl.graph((Original, Target))

    # 为节点特征赋值
    # 读取文件并分割每一行，然后转换成浮点数
    Nodes_Features = np.loadtxt(path, delimiter=' ')
    
    # print("Nodes_Features.shape:\n", Nodes_Features.shape, '\n')
    # print("Nodes_Features:\n", Nodes_Features, '\n')
                    
    graph.ndata['feat'] = torch.tensor(Nodes_Features)

    # 为边特征赋值
    graph.edata['feat'] = (Edges_Weight).reshape((Edges_Nums, 1))

    # 打印图信息
    # print((graph, Label))
    return (graph, Label)

In [160]:
import os

# 指定根路径
root_path = r'C:\Users\ChenXY\Desktop\VScode\Python\iGEM\Final_First\AllData'
# 指定要遍历的顶层子文件夹名称
top_subfolders = ['AdjacencyMatrix', 'OneHotEncodedData']
AdjacencyMatrix_FilePath = []
OneHotEncodedData_FilePath = []


# 遍历顶层子文件夹
for idx, top_folder in enumerate(top_subfolders):
    # 构造顶层子文件夹的路径
    top_path = os.path.join(root_path, top_folder)
    for root, dirs, files in os.walk(top_path):
        # print(root, dirs, files)
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                # print(root, "file_path:", file_path)
                if idx == 0:
                    AdjacencyMatrix_FilePath.append((root.split('\\')[-1], file_path))
                elif idx == 1:
                    OneHotEncodedData_FilePath.append((root.split('\\')[-1], file_path))

In [161]:
# 测试
Original, Target, Label, Nodes, Edges_Nums, Edges_Weight = MainDate(AdjacencyMatrix_FilePath[0][0], AdjacencyMatrix_FilePath[0][1])

GraphCreat(OneHotEncodedData_FilePath[0][1], Original, Target, Label ,Edges_Nums, Edges_Weight)

(Graph(num_nodes=58, num_edges=284,
       ndata_schemes={'feat': Scheme(shape=(25,), dtype=torch.float64)}
       edata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float64)}),
 tensor(0))

In [162]:
# # 测试
# Original, Target, Label, Nodes, Edges_Nums, Edges_Weight = MainDate(AdjacencyMatrix_FilePath[0][0], r"C:\Users\ChenXY\Desktop\VScode\Python\iGEM\Final_First\AllData\AdjacencyMatrix\label0\1156_AdjacencyMatrix.txt")
# # print(Original)
# # print(Target)
# # print(Label)
# # print(Edges_Nums)
# # print(Edges_Weight)
# GraphCreat(r"C:\Users\ChenXY\Desktop\VScode\Python\iGEM\Final_First\AllData\OneHotEncodedData\label0\1156_encoded.txt", Original, Target, Label ,Edges_Nums, Edges_Weight)

In [163]:
print(len(AdjacencyMatrix_FilePath), len(OneHotEncodedData_FilePath))

7386 7386


In [164]:
Dataset = []
for i in range(len(AdjacencyMatrix_FilePath)):
    if AdjacencyMatrix_FilePath[i][1].split('\\')[-2] == 'label0' and AdjacencyMatrix_FilePath[i][1].split('\\')[-1] == '1156_AdjacencyMatrix.txt':
        continue
    if AdjacencyMatrix_FilePath[i][1].split('\\')[-2] == 'label0' and AdjacencyMatrix_FilePath[i][1].split('\\')[-1] == '2271_AdjacencyMatrix.txt':
        continue
    if AdjacencyMatrix_FilePath[i][1].split('\\')[-2] == 'label1' and AdjacencyMatrix_FilePath[i][1].split('\\')[-1] == '513_AdjacencyMatrix.txt':
        continue
    if AdjacencyMatrix_FilePath[i][1].split('\\')[-2] == 'label1' and AdjacencyMatrix_FilePath[i][1].split('\\')[-1] == '616_AdjacencyMatrix.txt':
        continue
    # print(AdjacencyMatrix_FilePath[i][1])
    # print(OneHotEncodedData_FilePath[i][1])
    Original, Target, Label, Nodes, Edges_Nums, Edges_Weight = MainDate(AdjacencyMatrix_FilePath[i][0], AdjacencyMatrix_FilePath[i][1])
    (graph, Label) = GraphCreat(OneHotEncodedData_FilePath[i][1], Original, Target, Label ,Edges_Nums, Edges_Weight)
    Dataset.append((graph, Label))
    